In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
df = pd.read_csv("../VOD_EST/data/vod_cohorts.txt", sep="\t", header=None)

In [67]:
print(df.shape)
df.columns = ['encrypted_customer_Id','first_purchase','transaction_date','content_type']
df['transaction'] = 1

df['first_purchase'] = pd.to_datetime(df['first_purchase'])
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

df.head()

(4823959, 4)


,encrypted_customer_Id,first_purchase,transaction_date,content_type,transaction
0,A8XJNM5X8008M,2016-04-02,2016-04-04,Movie VOD,1
1,A365VJ4VMPBLC,2016-03-06,2016-06-13,Movie VOD,1
2,ATFVH4L38OZAF,2016-01-02,2016-08-05,Movie EST,1
3,A370OUC2FAMFF0,2016-09-23,2016-09-28,Movie VOD,1
4,A18MFDSY5MG1BL,2016-05-20,2016-10-28,Movie VOD,1


In [68]:
df['encrypted_customer_Id'].nunique()

483446

## Assign cohort: either VOD only or mixed

In [69]:
t = pd.pivot_table(df, index = 'encrypted_customer_Id', columns='content_type', values = 'transaction' , aggfunc = 'sum').reset_index()
t = t.fillna(0)

In [80]:
t.head()

content_type,encrypted_customer_Id,Movie EST,Movie VOD,class,total_units
0,A00025383DJZ2L6FHXPI0,0.0,2.0,VOD Only,3.0
1,A00033702UP7WZQ5KSDAW,0.0,2.0,VOD Only,3.0
2,A00035188K6CXZDRM4MC,4.0,2.0,Mixed,7.0
3,A000391421LY0FAFBU9ZH,0.0,46.0,VOD Only,47.0
4,A00041341E6PP1JSO0RKN,0.0,21.0,VOD Only,22.0


In [71]:
t['encrypted_customer_Id'].nunique()

483446

In [72]:
t['class'] = "NA"
t['class'] = np.where(t['Movie EST'] == 0 , 'VOD Only', t['class'])
t['class'] = np.where(t['Movie VOD'] == 0, 'Onlye_first_time_VOD', t['class'])
t['class'] = np.where((t['Movie VOD'] == 0) | (t['Movie EST'] > 0) , 'Mixed', t['class'])


In [73]:
t['class'].unique()

array(['VOD Only', 'Mixed'], dtype=object)

In [74]:
t.groupby("class").size()

class
Mixed       220173
VOD Only    263273
dtype: int64

In [75]:
t['total_units'] = t['Movie VOD'] + t['Movie EST'] + 1

In [76]:
t.groupby("class")['total_units'].sum()

class
Mixed       3735200.0
VOD Only    1572205.0
Name: total_units, dtype: float64

## Cohorts 
http://www.gregreda.com/2015/08/23/cohort-analysis-with-python/

In [77]:
#create period column based on transaction date
df['order_period'] = df['transaction_date'].apply(lambda x: x.strftime('%Y-%m'))

In [78]:
#determine user s cohort group
df['cohortgroup'] = df['first_purchase'].apply(lambda x : x.strftime('%Y-%m'))

In [79]:
df.head()

,encrypted_customer_Id,first_purchase,transaction_date,content_type,transaction,order_period,cohortgroup
0,A8XJNM5X8008M,2016-04-02,2016-04-04,Movie VOD,1,2016-04,2016-04
1,A365VJ4VMPBLC,2016-03-06,2016-06-13,Movie VOD,1,2016-06,2016-03
2,ATFVH4L38OZAF,2016-01-02,2016-08-05,Movie EST,1,2016-08,2016-01
3,A370OUC2FAMFF0,2016-09-23,2016-09-28,Movie VOD,1,2016-09,2016-09
4,A18MFDSY5MG1BL,2016-05-20,2016-10-28,Movie VOD,1,2016-10,2016-05


In [81]:
df_new = pd.merge(left=df,right=t,left_on='encrypted_customer_Id',right_on='encrypted_customer_Id', how="left")

In [84]:
df_new.shape
df_new.head()

,encrypted_customer_Id,first_purchase,transaction_date,content_type,transaction,order_period,cohortgroup,Movie EST,Movie VOD,class,total_units
0,A8XJNM5X8008M,2016-04-02,2016-04-04,Movie VOD,1,2016-04,2016-04,0.0,12.0,VOD Only,13.0
1,A365VJ4VMPBLC,2016-03-06,2016-06-13,Movie VOD,1,2016-06,2016-03,51.0,148.0,Mixed,200.0
2,ATFVH4L38OZAF,2016-01-02,2016-08-05,Movie EST,1,2016-08,2016-01,33.0,17.0,Mixed,51.0
3,A370OUC2FAMFF0,2016-09-23,2016-09-28,Movie VOD,1,2016-09,2016-09,38.0,133.0,Mixed,172.0
4,A18MFDSY5MG1BL,2016-05-20,2016-10-28,Movie VOD,1,2016-10,2016-05,4.0,295.0,Mixed,300.0
